# WULPUS Example: Waterbath Measurement

Cédric Hirschi

ETH Zurich, 2025

---

In this notebook, we show how you could analyze the data acquired during the waterbath experiment.

For this, we will need:
- The data file in `.npz` format
- The USS configuration file in `.json` format

For this example, you have the following files available:
- `examples/waterbath_0.npz`
- `examples/trx_waterbath.json`
- `examples/uss_waterbath.json`

## Preparation

First, we need to import the necessary libraries.

In [160]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.gridspec as pltgs
import numpy as np
import json
import scipy.signal as ss

Then, we load the data and the USS configuration.

In [ ]:
file = np.load('./examples/waterbath_0.npz')

data = file['data_arr'].T


data = data[file['tx_rx_id_arr'] == 0]

data.shape

In [ ]:
uss_config = json.load(open('./examples/uss_waterbath.json'))

frame_period = uss_config['meas_period'] / 1e6
sampling_freq = uss_config['sampling_freq']
central_freq = uss_config['pulse_freq']

assert uss_config['num_samples'] == data.shape[1]
assert uss_config['num_acqs'] == data.shape[0]

samples_per_frame = uss_config['num_samples']
num_frames = uss_config['num_acqs']

print('Measurement statistics:')
print(f'  {num_frames} frames at {1/frame_period:.2f} FPS')
print(f'  {samples_per_frame} samples per frame')
print(f'  {central_freq/1e6:.2f} MHz central frequency')
print(f'  {sampling_freq/1e6:.2f} MS/s sampling frequency')

## Analysis

Here, we define a helper function to plot the data and plot the raw data.

In [ ]:
def plot_waterbath(data, ref_times=[], lines=[]):

    fig = plt.figure(figsize=(12, 6))
    gs = fig.add_gridspec(1, 2)

    times = np.arange(num_frames) * frame_period
    depths = np.arange(samples_per_frame) / sampling_freq * 1540 / 2

    ax_mmode = fig.add_subplot(gs[0, 0])
    ax_mmode.imshow(data.T, aspect='auto', cmap='gray', extent=[0, times[-1], depths[-1]*1e2, 0])
    ax_mmode.set_xlabel('Time [s]')
    ax_mmode.set_ylabel('Depth [cm]')
    ax_mmode.set_title('M-Mode')
    

    for line in lines:
        ax_mmode.plot(times, depths[line]*1e2, 'r--')

    for ref_time in ref_times:
        ax_mmode.axvline(ref_time, color='g', linestyle='--')

    if ref_times:
        gs_ref_times = pltgs.GridSpecFromSubplotSpec(len(ref_times), 1, subplot_spec=gs[0, 1])

        for i, ref_time in enumerate(ref_times):
            ax = fig.add_subplot(gs_ref_times[i])

            ref_frame = int(ref_time / frame_period)

            ax.plot(depths*1e2, data[ref_frame], 'b')
            for line in lines:
                ax.plot(depths[line[ref_frame]]*1e2, data[ref_frame][line[ref_frame]], 'ro')

            ax.set_ylim(-2500, 2500)
            ax.grid()
            ax.annotate(f'Frame {ref_frame}', xy=(0.82, 0.1), xycoords='axes fraction')
            if i == 0:
                ax.set_title('Reference frames')
            if i == len(ref_times) - 1:
                ax.set_xlabel('Depth [cm]')
            else:
                ax.set_xticklabels([])

    plt.tight_layout()
    plt.show()

plot_waterbath(data)

### Reference Frames

From this raw plot, we can get some key reference frames, which we list here for use later in the analysis.

In [194]:
ref_times = [4.0, 5.15, 6.3, 7.45, 8.6]

### Filtering

Now, we filter the data. We only need the data around the central frequency, which is why we apply a bandpass filter.

This already gives us a clearer image of the data.

In [ ]:
def filter_data(data, central_freq, sampling_freq, filter_width=None, filter_order=5):
    if filter_width is None:
        filter_width = central_freq * 0.9
    b, a = ss.butter(filter_order, [central_freq - filter_width / 2, central_freq + filter_width / 2], btype='band', fs=sampling_freq)
    return ss.filtfilt(b, a, data, axis=1)

filtered_data = filter_data(data, central_freq, sampling_freq)

plot_waterbath(filtered_data, ref_times=ref_times)

### Envelope Extraction

To extract the envelope, we use the Hilbert transform. This gives us the envelope of the signal.

In [ ]:
def envelope_data(data):
    return np.abs(ss.hilbert(data, axis=1))

enveloped_data = envelope_data(filtered_data)

plot_waterbath(enveloped_data, ref_times=ref_times)

### Peak Detection

Finally, we can detect the peaks in the envelope for each frame. This gives us the distance of the metal object in the waterbath.

In [ ]:
MIN_DEPTH = 9e-3

def get_peak(data, min_depth=MIN_DEPTH):
    min_idx = int(min_depth * sampling_freq / 1540 * 2)

    return np.argmax(data[:, min_idx:], axis=1) + min_idx

maxima = get_peak(enveloped_data)

plot_waterbath(enveloped_data, ref_times=ref_times, lines=[maxima])

Since this distance is quite noisy, we apply a moving average filter to smooth the data.

In [ ]:
FILTER_WIDTH = 100e-3

def filter_maxima(maxima, filter_width=FILTER_WIDTH):
    filter_width = int(filter_width / frame_period) // 2 * 2 + 1
    return np.convolve(maxima, np.ones(filter_width) / filter_width, mode='same').astype(int)

maxima_filtered = filter_maxima(maxima)

plot_waterbath(enveloped_data, ref_times=ref_times, lines=[maxima_filtered])

We still get some artifact at the start of each frame. To omit this, we apply TGC (Time Gain Compensation) to the data.

In [ ]:
print(data)

In [ ]:
ALPHA = 1

def apply_gain(data, alpha=ALPHA):
    depths = np.arange(samples_per_frame) / sampling_freq * 1540 / 2
    gain = np.exp(alpha * depths)
    return data * gain

enveloped_data_gained = envelope_data(filter_data(apply_gain(data), central_freq, sampling_freq))
maxima_filtered_gained = filter_maxima(get_peak(enveloped_data_gained))

plot_waterbath(enveloped_data_gained, ref_times=ref_times, lines=[maxima_filtered_gained])

---
```text
   Copyright (C) 2025 ETH Zurich. All rights reserved.
   Author: Cedric Hirschi, ETH Zurich
   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at
       http://www.apache.org/licenses/LICENSE-2.0
   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

   SPDX-License-Identifier: Apache-2.0
```